In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
try:
    import sklearn
    print(sklearn.__version__)
    if (sklearn.__version__ < '1.3.0'): #update for latest scikit-learn version
        !pip install -U --user scikit-learn
except Exception as e:
    !pip install scikit-learn

In [ ]:
childcare = pd.read_csv("childcare_costs.csv")
childcare.head()

In [ ]:
counties = pd.read_csv("counties.csv")
counties.head()

In [ ]:
df = childcare.merge(counties, how = 'left', on = "county_fips_code")
df.head()

In [ ]:
nc_childcare = df[df['state_name'] == "North Carolina"]
nc_childcare

**Research Question 1**
Exploratory Data Analysis
1. What is the distribution of the relationship between child's stage and costs of childcare in North Carolina in the years 2014-2018?

In [ ]:
# Count the number of rows with at least one NaN
count_rows_with_nan = nc_childcare.isnull().any(axis=1).sum()
print("Number of rows with at least one NaN:", count_rows_with_nan)

In [ ]:
selected_columns = ['study_year','mcsa', 'mfccsa', 'mc_infant', 'mc_toddler', 'mc_preschool', 'mfcc_infant', 'mfcc_toddler', 'mfcc_preschool', 'county_name', 'state_name', 'state_abbreviation']
rq1 = nc_childcare[selected_columns]
rq1 = rq1.dropna(how = 'any')
rq1

In [ ]:
# In NC, aggregated median price charge for Family Childcare in the year 2014-2018
rq1_family = rq1.groupby('study_year')[['mfcc_infant', 'mfcc_toddler', 'mfcc_preschool', 'mfccsa']].mean()
rq1_family = rq1_family.reset_index()
rq1_family

In [ ]:
rq1_family['study_year'] = rq1_family['study_year'].astype(str)

# Plotting the data
ax = rq1_family.set_index('study_year').plot(kind='line', marker='o')

# Adding labels and title
plt.title('Median Price Increase for Family Childcare Groups Over Study Years')
plt.xlabel('Study Year')
plt.ylabel('Median Price')

# Adding legend
plt.legend(title='Childcare Groups')
ax.legend(['Infant', 'Toddler', 'Preschool', 'School Age'], title='Childcare Groups')

# Displaying the plot
plt.show()

Based on the distribution of the Family childcare spending of North Carolina in the years 2014 to 2018, we are able to make two general observations. 
1. There was a general increase in the spendings for all three child stages in the year 2014 to 2018.
2. Between the Child stages, families spend the most on childcare when the child was an infant (0-23 months), then when the child was a toddler(24-35 months), and lastly they spent the least when the child was in preschool(36-54 months). 

In [ ]:
# In NC, aggregated median price charge for Center-Based Childcare in the year 2014-2018
rq1_center = rq1.groupby('study_year')[['mc_infant', 'mc_toddler', 'mc_preschool', 'mcsa']].mean()
rq1_center = rq1_center.reset_index()
rq1_center

In [ ]:
rq1_center['study_year'] = rq1_center['study_year'].astype(str)

# Plotting the data
ax = rq1_center.set_index('study_year').plot(kind='line', marker='o')

# Adding labels and title
plt.title('Median Price Change for Center Childcare Groups Over Study Years')
plt.xlabel('Study Year')
plt.ylabel('Median Price')

# Adding legend
plt.legend(title='Childcare Groups')
ax.legend(['Infant', 'Toddler', 'Preschool', 'School Age'], title='Childcare Groups')

# Displaying the plot
plt.show()

In [ ]:
# Plotting specific columns from df1 (nc_year_grouped_center) in blue
plt.plot(rq1_center['study_year'], rq1_center['mc_infant'], label='mc_infant', color='blue', marker = 'o')
plt.plot(rq1_center['study_year'], rq1_center['mc_toddler'], label='mc_toddler', color='lightblue', marker = 's')
plt.plot(rq1_center['study_year'], rq1_center['mc_preschool'], label='mc_preschool', color='skyblue', marker = '^')
plt.plot(rq1_center['study_year'], rq1_center['mcsa'], label='mcsa', color='deepskyblue', marker = '+')

# Plotting specific columns from df2 (nc_year_grouped) in red
plt.plot(rq1_family['study_year'], rq1_family['mfcc_infant'], label='mfcc_infant', color='red', marker = 'o')
plt.plot(rq1_family['study_year'], rq1_family['mfcc_toddler'], label='mfcc_toddler', color='lightcoral', marker = 's')
plt.plot(rq1_family['study_year'], rq1_family['mfcc_preschool'], label='mfcc_preschool', color='salmon', marker = '^')
plt.plot(rq1_family['study_year'], rq1_family['mfccsa'], label='mfccsa', color='tomato', marker = '+')

# Adding labels and title
plt.title('Multiple Subsets of Lines with Different Colors')
plt.xlabel('Study Year')
plt.ylabel('Median Price')


# Display the plot
plt.show()

In [ ]:
rq1.head()# Group by study_year and get the unique counties for each year
counties = rq1.groupby('study_year')['county_name'].count()
counties
#basically, includes almost all counties in the last four years. 

**Hypothesis Test**

$H_0$: There is no difference between family and center-based childcare costs for any childcare group in 2018.

$H_a$: There is some difference between family and center-based childcare costs for any childcare group in 2018.

**Research Question**
2. What is the distribution of poverty rate/unemployment rate in North Carolina in the years 2014-2018? 

In [ ]:
nc_year_grouped_ec = nc_childcare.groupby('study_year')[['pr_f', 'pr_p', 'unr_16']].mean()
nc_year_grouped_ec = nc_year_grouped_ec.reset_index()
melted_data_ec = pd.melt(nc_year_grouped_ec, id_vars=['study_year'], value_vars=['pr_f', 'pr_p', 'unr_16'], var_name='Socioeconomic Indicators', value_name='Poverty/Unemployment Rate')
melted_data_ec
melted_data_ec = melted_data_ec[(melted_data_ec['study_year'] >= 2014) & (melted_data_ec['study_year'] <= 2018)]

g2 = sns.catplot(x='study_year', y = "Poverty/Unemployment Rate", hue = "Socioeconomic Indicators", kind='bar', data = melted_data_ec, palette='Paired', aspect=2)


g2.set_axis_labels("Study Year", "Poverty/Unemployment Rate")
new_labels = ['Poverty Rate for Families', 'Poverty Rate for Individuals', 'Unemployment Rate']
for t, l in zip(g2._legend.texts, new_labels):
    t.set_text(l)
    


# Add a title to the plot
plt.title("Poverty/Unemployment rate in North Carolina in the years 2014-2018")

# Show the plot
plt.show()

Based on the distribution of the Poverty/Unemployment Rate of North Carolina in the years 2014 to 2018, we are able to make two general observations.

1. There was a general decrease in the Poverty and the Unemployment rates in the year 2014 to 2018.
2. Between the different socioeconomic indicators, poverty rate for individuals was the highest, followed by the poverty rate for families. Lastly, the unemployment rate was the lowest. 

**Research Question 3**
What is the relationship between child care cost and average income in the year 2018 in North Carolina?

In [ ]:
nc_year_grouped = nc_childcare.groupby('mhi_2018')[['mfcc_infant', 'mfcc_toddler', 'mfcc_preschool']].mean()
nc_year_grouped = nc_year_grouped.reset_index()
melted_data = pd.melt(nc_year_grouped, id_vars=['mhi_2018'], value_vars=['mfcc_infant', 'mfcc_toddler', 'mfcc_preschool'], var_name='Child Stage', value_name='Mean of Median Price Charge for Family Childcare')
plt.figure(figsize=(14, 8))
g3 = sns.scatterplot(x='mhi_2018', y="Mean of Median Price Charge for Family Childcare", hue="Child Stage",palette='Accent',data=melted_data,s=20)
plt.title('Median Household Income and Average Price for Childcare in 2018')
plt.xlabel('Median Household Income 2018')
plt.ylabel('Average Price Charge for Family Childcare')

new_labels = ['Infant', 'Toddler', 'Preschool']
g3.get_legend().set_title("Child Stage")
for t, l in zip(g3.get_legend().texts, new_labels):
    t.set_text(l)
    
plt.show()    

Based on the scatterplot of the relationship between child care cost and average income in the year 2018 in North Carolina, we are able to make two general observations.

1. There is a positive relationship between the median household income and average price charge for family childcare cost in 2018.
2. Between the different child stages, families spend the most on childcare when the child was an infant (0-23 months), then when the child was a toddler(24-35 months), and lastly they spent the least when the child was in preschool(36-54 months).

# Linear regression models

In [ ]:
## only select 2018 for linear regression, prepare data for model by replacing NAs
nc_childcare_2018 = nc_childcare[nc_childcare["study_year"] == 2018]
nc_childcare_2018 = nc_childcare_2018.dropna(subset=["mcsa"])

### Predicting median childcare cost for schoolage children given median income, unemployment rate, and poverty rate:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
## this linear model is supposed to use the predictors in household_data to predict median childcare for schoolaged children

linear_model = LinearRegression()
# unemployment rate, poverty rate for families, median household income
household_data = nc_childcare_2018[["unr_16", "pr_f", "mhi_2018"]].values
# mcsa --> "Weekly, full-time median price charged for Center-Based Care for those who are school age based on the results reported in the market rate survey report for the county or the rate zone/cluster to which the county is assigned."
target = nc_childcare_2018["mcsa"].values
linear_model.fit(X = household_data, y = target)

nc_childcare_2018["predicted_childcare_cost_schoolage"] = linear_model.predict(household_data)

nc_childcare_2018["baseline"] = nc_childcare_2018["mcsa"].median()

mse = mean_squared_error(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_schoolage"].values)
r2 = r2_score(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_schoolage"].values)
mse_base = mean_squared_error(nc_childcare_2018["mcsa"].values, nc_childcare_2018["baseline"].values)
r2_base = r2_score(nc_childcare_2018["mcsa"].values, nc_childcare_2018["baseline"].values)

print("MSE model", mse, "R2 model", r2)
print("MSE baseline", mse_base, "R2 base", r2_base)

In [ ]:
## can conclude that including median household income does improve the accuracy of the model

## this linear model is supposed to use the predictors in household_data to predict median childcare for schoolaged children

linear_model = LinearRegression()
# unemployment rate, poverty rate for families, median household income
household_data = nc_childcare_2018[["unr_16", "pr_f"]].values
# mcsa --> "Weekly, full-time median price charged for Center-Based Care for those who are school age based on the results reported in the market rate survey report for the county or the rate zone/cluster to which the county is assigned."
target = nc_childcare_2018["mcsa"].values
linear_model.fit(X = household_data, y = target)

nc_childcare_2018["predicted_childcare_cost_schoolage"] = linear_model.predict(household_data)

nc_childcare_2018["baseline"] = nc_childcare_2018["mcsa"].median()

mse = mean_squared_error(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_schoolage"].values)
r2 = r2_score(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_schoolage"].values)
mse_base = mean_squared_error(nc_childcare_2018["mcsa"].values, nc_childcare_2018["baseline"].values)
r2_base = r2_score(nc_childcare_2018["mcsa"].values, nc_childcare_2018["baseline"].values)
print("*mse and r2 for model WITHOUT median household income*")
print("MSE model,", mse, "R2 model", r2)

#### Conclusions from model:

Our first model appears to explain about 26.82% of the variability in our data, with a MSE of 212.17. **good or bad??**. We next removed the "mhi_2018" variable, which represents median household income, as a predictor in our model. This yielded a lower R^2 of about 0.1260, or 12.60%, and a higher MSE of 254.07. Thus, including median household income as a predictor of median household income improved our model.

### Predicting median childcare cost for schoolage children given number of household, county percentage hispanic, and median household income

In [ ]:
## this linear model is supposed to use the predictors in household_data to predict median childcare for schoolaged children

linear_model = LinearRegression()
# unemployment rate, poverty rate for families, median household income
household_data = nc_childcare_2018[["hispanic", "mhi_2018", "households"]].values
# mcsa --> "Weekly, full-time median price charged for Center-Based Care for those who are school age based on the results reported in the market rate survey report for the county or the rate zone/cluster to which the county is assigned."
target = nc_childcare_2018["mcsa"].values
linear_model.fit(X = household_data, y = target)

nc_childcare_2018["predicted_childcare_cost_byhispanic"] = linear_model.predict(household_data)

nc_childcare_2018["baseline"] = nc_childcare_2018["mcsa"].median()

mse = mean_squared_error(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_byhispanic"].values)
r2 = r2_score(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_byhispanic"].values)
mse_base = mean_squared_error(nc_childcare_2018["mcsa"].values, nc_childcare_2018["baseline"].values)
r2_base = r2_score(nc_childcare_2018["mcsa"].values, nc_childcare_2018["baseline"].values)

print("MSE model", mse, "R2 model", r2)
print("MSE baseline", mse_base, "R2 base", r2_base)

In [ ]:
linear_model = LinearRegression()
# unemployment rate, poverty rate for families, median household income
household_data = nc_childcare_2018[["mhi_2018", "households"]].values
# mcsa --> "Weekly, full-time median price charged for Center-Based Care for those who are school age based on the results reported in the market rate survey report for the county or the rate zone/cluster to which the county is assigned."
target = nc_childcare_2018["mcsa"].values
linear_model.fit(X = household_data, y = target)

nc_childcare_2018["predicted_childcare_cost_byhispanic"] = linear_model.predict(household_data)

nc_childcare_2018["baseline"] = nc_childcare_2018["mcsa"].median()

mse = mean_squared_error(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_byhispanic"].values)
r2 = r2_score(nc_childcare_2018["mcsa"].values, nc_childcare_2018["predicted_childcare_cost_byhispanic"].values)

print("*MSE and R2 without hispanic predictor*")
print("MSE model", mse, "R2 model", r2)

#### Conclusions from model:

Our first model appears to explain about 32.77% of the variability in our data, with a MSE of 195.42. **good or bad??**. We next removed the "hispanic" variable, which median household income, as a predictor in our model. This yielded a lower R^2 of about 0.1260, or 12.60%, and a higher MSE of 254.07. Thus, including median household income as a predictor of median household income improved our model.